In [1]:
import os
from mongo_connect import load_client
import pandas as pd 
from pandas_gbq import to_gbq
import google.auth

client = load_client()
CURR_DIR = os.getcwd()

db = client.get_database('Cluster0')
col = db.get_collection('games_rating')





In [2]:
import datetime
import calendar

curr_month = datetime.datetime.now().month
curr_day = datetime.datetime.now().day

months = []
for i in range(7): 
    months.append(curr_month)
    if curr_month != 1:
        curr_month -= 1
    else:
        curr_month = 12

day_count = []
for i in months: 
    num_days = calendar.monthrange(datetime.datetime.now().year, i)[1]
    day_count.append(num_days)

last_year = months[0] < months[-1]

if last_year: 
    six_mounth_ago = datetime.datetime(datetime.datetime.now().year - 1, months[-1], datetime.datetime.now().day)
else:
    six_mounth_ago = datetime.datetime(datetime.datetime.now().year, months[-1], datetime.datetime.now().day)

# gerer le cas ou le jour est 31 ou 30 et n'existe pas dans le mois d'il ya 6mois

six_mounth_ago_unix = six_mounth_ago.timestamp()



In [3]:
result = col.find({'unixReviewTime': {'$lt': six_mounth_ago_unix}})

documents = list(result)
df = pd.DataFrame(documents)


In [4]:
df = df.drop(columns=['image', 'style', 'reviewText', 'summary'])

df.head()

,_id,overall,verified,reviewTime,reviewerID,asin,reviewerName,unixReviewTime,vote
0,65ef2deecc71a763989a5c64,5.0,True,"10 17, 2015",A1HP7NVNPFMA4N,0700026657,Ambrosia075,1445040000,NaN
1,65ef2deecc71a763989a5c65,4.0,False,"07 27, 2015",A1JGAP0185YJI6,0700026657,travis,1437955200,NaN
2,65ef2deecc71a763989a5c66,3.0,True,"02 23, 2015",A1YJWEXHQBWK2B,0700026657,Vincent G. Mezera,1424649600,NaN
3,65ef2deecc71a763989a5c67,2.0,True,"02 20, 2015",A2204E1TH211HT,0700026657,Grandma KR,1424390400,NaN
4,65ef2deecc71a763989a5c68,5.0,True,"12 25, 2014",A2RF5B5H74JLPE,0700026657,jon,1419465600,NaN


In [5]:
df['average_overall'] = df.groupby(by='asin')['overall'] \
            .transform('mean')
           
df.tail()

,_id,overall,verified,reviewTime,reviewerID,asin,reviewerName,unixReviewTime,vote,average_overall
497572,65ef335ccc71a76398a1f408,4.0,True,"07 1, 2017",AVECM71LSZLC5,B01HGPUTCA,boris teplitskiy,1498867200,NaN,4.600000
497573,65ef335ccc71a76398a1f409,3.0,True,"08 20, 2018",A1RS06313BL6WN,B01HH6JEOC,Tom Stopsign,1534723200,NaN,3.625000
497574,65ef335ccc71a76398a1f40a,3.0,True,"08 7, 2017",ACIZ77IGIX2JL,B01HH6JEOC,Era,1502064000,NaN,3.625000
497575,65ef335ccc71a76398a1f40b,4.0,False,"08 5, 2018",A34GG58TJ1A3SH,B01HIZF7XE,seamonkey10,1533427200,NaN,4.574468
497576,65ef335ccc71a76398a1f40c,2.0,True,"03 13, 2018",A6W81WTFK940B,B01HIZGKOE,msam420,1520899200,NaN,3.500000


In [6]:
df.groupby('asin').head()

,_id,overall,verified,reviewTime,reviewerID,asin,reviewerName,unixReviewTime,vote,average_overall
0,65ef2deecc71a763989a5c64,5.0,True,"10 17, 2015",A1HP7NVNPFMA4N,0700026657,Ambrosia075,1445040000,NaN,3.692308
1,65ef2deecc71a763989a5c65,4.0,False,"07 27, 2015",A1JGAP0185YJI6,0700026657,travis,1437955200,NaN,3.692308
2,65ef2deecc71a763989a5c66,3.0,True,"02 23, 2015",A1YJWEXHQBWK2B,0700026657,Vincent G. Mezera,1424649600,NaN,3.692308
3,65ef2deecc71a763989a5c67,2.0,True,"02 20, 2015",A2204E1TH211HT,0700026657,Grandma KR,1424390400,NaN,3.692308
4,65ef2deecc71a763989a5c68,5.0,True,"12 25, 2014",A2RF5B5H74JLPE,0700026657,jon,1419465600,NaN,3.692308
...,...,...,...,...,...,...,...,...,...,...
497555,65ef335ccc71a76398a1f3f7,2.0,True,"03 13, 2018",A2WPRZ46BN54GU,B01HD1B76O,Jason A Gallant,1520899200,NaN,1.428571
497559,65ef335ccc71a76398a1f3fb,5.0,True,"01 25, 2018",A4MSCKHWVOAZF,B01HD2TECW,Edward D. Wall,1516838400,15,4.800000
497560,65ef335ccc71a76398a1f3fc,4.0,True,"08 6, 2017",AOHG53ZO6BJ7R,B01HD2TECW,Joe,1501977600,20,4.800000
497571,65ef335ccc71a76398a1f407,5.0,True,"07 1, 2017",A2NKYXCAGBGW7F,B01HGPUTCA,Snipun1,1498867200,NaN,4.600000


In [7]:
df['game_id'] = df['asin'] 
df['avg_note'] = df['average_overall']
df['user_note'] = df['overall']
df['latest_note'] = df.groupby('asin')['unixReviewTime'].transform('max').astype(int)
df['oldest_note'] = df.groupby('asin')['unixReviewTime'].transform('min').astype(int)

to_drop = [col for col in df.columns if col not in ['game_id', 'avg_note', 'user_note', 'latest_note', 'oldest_note']]

print(to_drop)
df.drop(columns=to_drop, inplace=True)
df.head()



['_id', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin', 'reviewerName', 'unixReviewTime', 'vote', 'average_overall']


,game_id,avg_note,user_note,latest_note,oldest_note
0,0700026657,3.692308,5.0,1519689600,1337040000
1,0700026657,3.692308,4.0,1519689600,1337040000
2,0700026657,3.692308,3.0,1519689600,1337040000
3,0700026657,3.692308,2.0,1519689600,1337040000
4,0700026657,3.692308,5.0,1519689600,1337040000


In [8]:
df['user_note'] = df['user_note'].astype(int)

df['game_id'] = (df['game_id'].astype('category').cat.codes + 1).astype(int)



In [11]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "google_credentials.json"

credentials, project_id = google.auth.default()

to_gbq(df, 'dengineer-413113.datalake.video_games', project_id='dengineer-413113', credentials=credentials)